In [3]:
#곱셈 계층 MulLayer
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y
    
        return out
    def backward(self,dout): # 상류에서 넘어온 미분(dout)에 순전파 때의 값을 서로 바꿔 곱한 후 하류로 흘림
        
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy
    
    
apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

#역전파
dprice = 1 # 출력층의 미분값은 1
dapple_price, tax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num)

220.00000000000003
2.2 110.00000000000001


In [4]:
#덧셈계층 AddLayer
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y):
        out = x+y
        return out
    def backward(self,dout):
        dx = dout *1#덧셈노드는 그대로이기 때문에
        dy = dout *1
        return dx,dy
    

apple = 100
apple_num =2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange,orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#역전파 
dprice = 1 #맨 끝에서 출발할땐 미분값이 1이기 때문에 
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

    

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [7]:
#활성화 함수 계층 구현하기 
# ReLU 계층!!!
#신경망을 구성하는 층인 계층은 각각을 클래스 하나로 구현
import tensorflow as tf
import numpy as np
class Relu:
    def __init__(self):
        self.mask = None###mask는 0보다 크면 False로 작으면 True로 형태로 배열 저장
    def foward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
    def backward(self,dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
x = np.array([[1,-0.5],[-2,3]])
print(x)

mask = (x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [8]:
#Sigmoid계층
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1 /(1+np.exp(-x))
        self.out = out
        
        return out
    def backward(self,dout):
        dx = dout*(1-self.out)*self.out
        return dx
    ## 순전파의 출력을 out에다가 보관했다가 다시 역전파 계신시 그 값을 이용
    ###즉 dout * y(1-y)!!
    

In [12]:
#배치용 Affine계층
x_dot_W = np.array([[0,0,0],[10,10,10]])
B = np.array([1,2,3])

print(x_dot_W)

print(x_dot_W + B)
## 이렇게 되면 안됨 첫번째 열의 합이 더해져야함
dy = np.array([[1,2,3],[4,5,6]])
print(dy)

db = np.sum(dy,axis=0)
print(db)
##이렇게 해야 됨 

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]
[[1 2 3]
 [4 5 6]]
[5 7 9]


In [ ]:
##Affine 계층 클래스 


class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W)+self.B
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout,self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
    
    